<a href="https://colab.research.google.com/github/MateoProjects/IA/blob/main/Aprenentatge_per_refor%C3%A7_cap9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Sistemes value-based: Deep Q-Network


**Nota:** S'ha d'executar primer aquest script per tal de poder obtenir els jocs per l'entorn.

In [3]:
! wget http://www.atarimania.com/roms/Roms.rar
! mkdir /content/ROM/
! unrar e /content/Roms.rar /content/ROM/
! python -m atari_py.import_roms /content/ROM/


--2021-08-22 12:23:43--  http://www.atarimania.com/roms/Roms.rar
Resolving www.atarimania.com (www.atarimania.com)... 195.154.81.199
Connecting to www.atarimania.com (www.atarimania.com)|195.154.81.199|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11128004 (11M) [application/x-rar-compressed]
Saving to: ‘Roms.rar’

Roms.rar            100%[===================>]  10.61M   474KB/s    in 24s     

2021-08-22 12:24:08 (457 KB/s) - ‘Roms.rar’ saved [11128004/11128004]


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from /content/Roms.rar

Extracting  /content/ROM/HC ROMS.zip                                      36%  OK 
Extracting  /content/ROM/ROMS.zip                                         74% 99%  OK 
All OK
copying adventure.bin from ROMS/Adventure (1980) (Atari, Warren Robinett) (CX2613, CX2613P) (PAL).bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/adventure.bin
copying air_raid.bin fr

In [4]:
!python -m atari_py.import_roms .

copying adventure.bin from ROMS/Adventure (1980) (Atari, Warren Robinett) (CX2613, CX2613P) (PAL).bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/adventure.bin
copying air_raid.bin from ROMS/Air Raid (Men-A-Vision) (PAL) ~.bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/air_raid.bin
copying alien.bin from ROMS/Alien (1982) (20th Century Fox Video Games, Douglas 'Dallas North' Neubauer) (11006) ~.bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/alien.bin
copying amidar.bin from ROMS/Amidar (1982) (Parker Brothers, Ed Temple) (PB5310) ~.bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/amidar.bin
copying assault.bin from ROMS/Assault (AKA Sky Alien) (1983) (Bomb - Onbase) (CA281).bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/assault.bin
copying asterix.bin from ROMS/Asterix (AKA Taz) (07-27-1983) (Atari, Jerome Domurat, Steve Woita) (CX2696) (Prototype).bin to /usr/local/lib/python3.7/dist-packages/a

In [5]:
!nvidia-smi


Sun Aug 22 12:24:23 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
import gym
import torch
import torch.nn as nn        
import torch.optim as optim 
ENV_NAME = "PongNoFrameskip-v4" 
test_env = gym.make(ENV_NAME)
print(test_env.action_space.n)

6


In [7]:
print(test_env.observation_space.shape)

(210, 160, 3)


Ens està dient que fa 210 d'altura per 160 amplada amb 128 colors. 

A causa que té molts pixels seguint l'article DQN es fa un processament previ on es transformarà el joc a una matriu de 84x84 i es passa de colors a escala de grisos. Després, com amb una imatge no es pot determinar la direcció de la pilota el que es fa en aquest exemple és agrupar de 4 en 4 frames per veure cap a on va. 

Previament es calculava una sola en cada time step  ara es processarà un valor Q per totes les accions possibles en una sola pasada. 

In [8]:
def make_DQN(input_shape, output_shape):
    net = nn.Sequential(
        nn.Conv2d(input_shape[0], 32, kernel_size=8, stride=4),
        nn.ReLU(),
        nn.Conv2d(32, 64, kernel_size=4, stride=2),
        nn.ReLU(),
        nn.Conv2d(64, 64, kernel_size=3, stride=1),
        nn.ReLU(),
        nn.Flatten(),
        nn.Linear(64*7*7, 512),
        nn.ReLU(),
        nn.Linear(512, output_shape)
    )
    return net

    
device = torch.device("cuda")
test_net = make_DQN(test_env.observation_space.shape, test_env.action_space.n).to(device)


In [11]:
print(test_env.unwrapped.get_action_meanings())

['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']


Per tal d'iniciar el joc se li ha de donar al botó Fire. Es crea un wrapper que iniciarà el joc. El reset el segon if està pensat per altres jocs. En el nostre cas amb el primer ja aniriem bé. 

Es declarà la classe FireResetEnv que permetrà fer el reset e iniciar el joc novament. 

In [12]:
class FireResetEnv(gym.Wrapper):
  def __init__(self, env=None):
    super(FireResetEnv, self).__init__(env)
    assert env.unwrapped.get_action_meanings()[1] == 'FIRE'
    assert len(env.unwrapped.get_action_meanings()) >= 3

  def step(self,action):
    return self.env.step(action)
  
  def reset(self):
    self.env.reset()
    obs, _, done, _ = self.env.step(1)
    if done:
      self.env.reset()
    obs, _, done, _ = self.env.step(2)
    if done:
      self.env.reset()
    return obs

Es codifiquen transformacions per el joc

In [14]:
import numpy as np

class MaxAndSkipEnv(gym.Wrapper):
  def __init__(self, env= None, skip=4):
    super(MaxAndSkipEnv,self).__init__(env)
    self._obs_buffer = collections.deque(maxlen=2)
    self._skip = skip


  def step(self,action):
    
    total_reward = 0.0
    done = None
    for _ in range(self._skip):
      obs, reward, done, info = self.env.step(action)
      self._obs_buffer.append(obs)
      total_reward += total_reward
      if done: 
        break
    max_frame = np.max(np.stack(self._obs_buffer), axis=0)
  
  def reset(self):
    self._obs_buffer.clear()
    obs = self.env.reset()
    self._obs_buffer.append(obs)
    return obs

Transformació de l'imatge de 210x160 a 84x84


In [ ]:
class ProcessFrame84(gym.ObservationWrapper):
  def __init__(self, env=None):
    super(ProcessFrame84, self).__init__(env)
    self.observation_space = gym.spaces.Box(low=0, high=255, shape=(84,84,1), dtype=uint8)

  def observation(self,obs):
    return ProcessFrame84.process(obs)

  @staticmethod
  def process(frame):
    if frame.size == 210*160*3:
      img
       = np.reshape(frame, [210,160,3]).astype(np.float32)
  
  img = img[:, :, 0]*0.299 + img[:, :, 1]*0.587 + img[:, :, 2]*0.114